In [12]:
import pandas as pd
from bokeh.plotting import figure
from nobody.utils import load_hist, load_all_hist
from nobody.utils.utils import get_ts_client
from nobody.settings import config
from bokeh.plotting import figure, output_notebook, show

ts = get_ts_client()


In [52]:
feed = load_all_hist()

In [28]:
# data = ts.pro_bar(ts_code="000001.SZ", adj='qfq', start_date="2018-01-01")
sh_index = ts.pro_bar(ts_code='000001.SH', asset='I', start_date=config["START_DATE"])
sh_index.index = pd.to_datetime(sh_index.trade_date)
sh_index.sort_index(inplace=True)
sh_index.head()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,,
2012-01-04,000001.SH,20120104,2169.390,2211.995,2217.520,2168.644,2199.417,-30.027,-1.3652,49245537.0,4.063292e+07
2012-01-05,000001.SH,20120105,2148.452,2160.898,2183.404,2145.556,2169.390,-20.938,-0.9652,58743447.0,4.660397e+07
2012-01-06,000001.SH,20120106,2163.395,2148.147,2164.322,2132.626,2148.452,14.943,0.6955,50583054.0,3.946587e+07
2012-01-09,000001.SH,20120109,2225.890,2164.741,2226.218,2148.453,2163.395,62.495,2.8887,76759839.0,6.213296e+07
2012-01-10,000001.SH,20120110,2285.744,2221.832,2288.628,2218.275,2225.890,59.854,2.6890,109585415.0,9.104590e+07


In [60]:
feed["000001.SZ"].tail()
'2019-07-24' not in feed["000001.SZ"].index

False

In [79]:
p_index = figure(title="trend", plot_width=800, plot_height=250, x_axis_type="datetime")

p_index.line(data.index, data.close, color='navy', alpha=0.5)
output_notebook()
show(p_index)

Loading BokehJS ...

In [55]:
len(feed)

3659

In [67]:
# latest 10 days up/down bars
sh_index.index[-10:]


l10 = {}
l10["x_range"] = [dt.strftime("%Y-%m-%d") for dt in sh_index.index[-10:]]
l10["limit_up"] = []
l10["limit_down"] = []
print(l10["x_range"])

for dt in l10["x_range"]:
    print(dt)
    limit_up_count = 0
    limit_down_count = 0

    for code, hist in feed.items():
        if dt not in hist.index:
            continue
        
        df = hist.loc[dt]
        if isinstance(df, pd.core.series.Series):
            if df.pct_chg > 9.5:
                limit_up_count += 1
            elif df.pct_chg < -9.5:
                limit_down_count += 1

    l10["limit_up"].append(limit_up_count)
    l10["limit_down"].append(limit_down_count)

print(l10)

['2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-29', '2019-07-30', '2019-07-31', '2019-08-01', '2019-08-02', '2019-08-05']
2019-07-23
2019-07-24
2019-07-25
2019-07-26
2019-07-29
2019-07-30
2019-07-31
2019-08-01
2019-08-02
2019-08-05
{'x_range': ['2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26', '2019-07-29', '2019-07-30', '2019-07-31', '2019-08-01', '2019-08-02', '2019-08-05'], 'limit_up': [49, 53, 36, 34, 43, 38, 29, 36, 30, 46], 'limit_down': [21, 0, 1, 10, 4, 3, 4, 3, 4, 8]}


In [80]:

from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge


fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

cds = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=l10)

p_dist = figure(x_range=l10["x_range"], y_range=(0, max(l10["limit_down"] + l10["limit_up"]) + 12), plot_height=350, plot_width=800, title="Date",
           toolbar_location=None, tools="")

p_dist.vbar(x=dodge('x_range', -0.25, range=p_dist.x_range), top='limit_down', width=0.2, source=source,
       color="#2ca25f", legend=value("跌停"))

p_dist.vbar(x=dodge('x_range',  0.0,  range=p_dist.x_range), top='limit_up', width=0.2, source=source,
       color="#e84d60", legend=value("涨停"))

p_dist.x_range.range_padding = 0.1
p_dist.xgrid.grid_line_color = None
p_dist.legend.location = "top_right"
p_dist.legend.orientation = "horizontal"

show(p_dist)



In [69]:
from bokeh.plotting import gridplot
import numpy as np

pct_chg_lst = []
dt = "2019-08-05"

for code, hist in feed.items():
    if dt not in hist.index:
        continue
    
    df = hist.loc[dt]
    pct_chg_lst.append(df.pct_chg)

hist1, edges1 = np.histogram(pct_chg_lst, bins=20)
print(hist1)

[  10   62  996 2170  293   59   12   42    1    2    0    1    0    0
    0    0    0    0    0    1]


In [81]:
p_hist = figure(title="dist", plot_width=800, plot_height=400, y_range=(0, max(hist1)))

p_hist.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
       fill_color="navy", line_color="white", alpha=0.5)


show(p_hist)
# show(p1)

In [91]:
from bokeh.io import save, output_file
from bokeh.layouts import Row
output_file("demo.html")
p_hist = figure(title="dist", plot_width=800, plot_height=400, y_range=(0, max(hist1)))

p_hist.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:],
       fill_color="navy", line_color="white", alpha=0.5)

show(p_hist)

In [92]:
show(gridplot([[p_index], [p_hist, p_dist]]))